In [107]:
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore") # ignore warning from libraries 
import math 
from six.moves import urllib
import pandas as pd
import datetime
from talib.abstract import *
import talib
import pinkfish as pf
from tqdm import tqdm
import plotly.graph_objects as go
import datetime
import plotly.express as px

In [94]:
# inmporting frequency and sentiment table
data = pd.read_csv('/Users/ohmat/Documents/Project/scrapingreddit/weekly_frequencies.csv')
# creating list of available tickers
tickers = data['Term'].tolist()

In [ ]:
# function that is used to finalize timeseries for trading in the timespan we observed
def finalize(timeseries):
    start = datetime.datetime(2021, 5, 10)
    end = datetime.datetime(2021, 5, 18)
    timeseries = pf.select_tradeperiod(timeseries, start, end, use_adj=True) # select tradeperiod from original timeseries
    timeseries, start = pf.finalize_timeseries(timeseries, start)
    return timeseries

In [65]:
# getting price timeseries for evey ticker in the weekly list
timeseries = []
for ticker in tqdm(tickers):
        globals()['ts_' + str(ticker)] = pf.fetch_timeseries(ticker)# call prices via pinkfish yahoo api and store in a global variable that consits of ts_ticker
        timeseries.append('ts_' + str(ticker)) # append them to a list for easier mass wrangling later

100%|██████████| 210/210 [00:01<00:00, 150.35it/s]


In [111]:
data

,Term,frequency10_05,Company_Name_x,Sector_x,frequency11_05,polarity11_05,frequency12_05,polarity12_05
0,NEW,78.0,0,Miscellaneous,0.0,0.000000,0.0,0.000000
1,TECH,49.0,Bio-Techne Corp Common Stock,Health Care,108.0,-0.312500,108.0,-0.312500
2,PLTR,134.0,Palantir Technologies Inc. Class A Common Stock,Technology,270.0,0.025167,270.0,0.025167
3,PLAY,40.0,0,Consumer Services,0.0,0.000000,0.0,0.000000
4,LOVE,19.0,The Lovesac Company Common Stock,0,67.0,0.166667,67.0,0.166667
...,...,...,...,...,...,...,...,...
205,OXY,0.0,Occidental Petroleum Corporation Common Stock,Energy,4.0,0.000000,4.0,0.000000
206,ACCD,0.0,Accolade Inc. Common Stock,0,1.0,0.091886,1.0,0.091886
207,CNC,0.0,Centene Corporation Common Stock,Health Care,1.0,0.000000,1.0,0.000000
208,NCLH,0.0,Norwegian Cruise Line Holdings Ltd. Ordinary S...,Transportation,1.0,0.000000,1.0,0.000000


In [115]:
#df.max().nlargest(10)
fig = px.bar(data, x='Term', y='frequency10_05')
fig.show()

In [ ]:
data['frequency10_05']

In [66]:
# creating a timeseries for index comparison, the S&p500
ts_SPX = pf.fetch_timeseries('SPY') 

In [106]:
# Finalizing timeseries, since we are only interested in prices from short before the analysis
ts_SPX = finalize(ts_SPX)
ts_GME=finalize(ts_GME)

In [105]:
# show chart for index
fig = go.Figure(data=go.Ohlc(x=ts_SPX.index,
                    open=ts_SPX['open'],
                    high=ts_SPX['high'],
                    low=ts_SPX['low'],
                    close=ts_SPX['close']))
fig.show()

In [110]:
fig = px.line(ts_GME, x=ts_GME.index, y="close", title='GME Price')
fig.show()